In [1]:
from itertools import product
from mip import Model, xsum, minimize, BINARY

In [2]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 20.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.9 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install openpyxl

In [21]:
import os
import pandas as pd
import re
os.chdir('/Users/chenya68/Documents/GitHub/uo-mip')
df_raw = pd.read_excel('data/OSD_schedules.xlsx',sheet_name = 2)
df_raw.columns = [re.sub('[^A-Za-z0-9Δ]+', '_', element) for element in df_raw.columns]
df_raw.fillna(0, inplace=True)
df_raw

,process_index,process_name,processing_time_weeks_,engineers,rooms,LM_40_VBE_1_BN_,LM_40_VBE_2_BN_,LM_40_Isolator_BN_,PTM_300_BN_,Sieve,Spray_Dryer_,Film_Coater_
0,0,start,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,MK-8591-C,7,1,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,MK-1167-B,4,1,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0
3,3,MK-3543-B,4,1,1,1.0,0.0,1.0,1.0,1.0,0.0,0.0
4,4,MK-3543-A1,1,1,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0
5,5,MK-3543-A2,1,1,1,1.0,0.0,0.0,1.0,1.0,1.0,0.0
6,6,MK-1084-D,5,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,7,MK-1084-A1,4,1,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0
8,8,MK-1084-H,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,9,EP-200-C,3,1,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [23]:
n = len(df_raw) - 2 # note there will be exactly 11 jobs (n=9 jobs plus the two 'dummy' ones)

p = df_raw['processing_time_weeks_'].to_list()

u = df_raw.iloc[:,3:].to_numpy().tolist()
#print(u)

c = [11,11,3,1,1,2,2,1,1]

S = [[0, 1], [0, 2], [0, 3], [0,6], [0,9],
     [3,4],[4,5],[6,7],[7,8],
      [1, 10], [2, 10],[5, 10], [8, 10],[9,10]]

(R, J, T) = (range(len(c)), range(len(p)), range(sum(p)))

In [24]:
model = Model()

x = [[model.add_var(name="x({},{})".format(j, t), var_type=BINARY) for t in T] for j in J]

model.objective = xsum(t * x[n + 1][t] for t in T)

for j in J:
    model += xsum(x[j][t] for t in T) == 1

for (r, t) in product(R, T):
    model += (
        xsum(u[j][r] * x[j][t2] for j in J for t2 in range(max(0, t - p[j] + 1), t + 1))
        <= c[r])

for (j, s) in S:
    model += xsum(t * x[s][t] - t * x[j][t] for t in T) >= p[j]

model.optimize()

print("Schedule: ")
for (j, t) in product(J, T):
    if x[j][t].x >= 0.99:
        print("Job {}: begins at t={} and finishes at t={}".format(j, t, t+p[j]))
print("Makespan = {}".format(model.objective_value))

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 3490 small elements
Coin0506I Presolve 234 (-61) rows, 328 (-2) columns and 4051 (-1261) elements
Clp1000I sum of infeasibilities 4.15133e-05 - average 1.77407e-07, 8 fixed columns
Coin0506I Presolve 234 (0) rows, 320 (-8) columns and 3908 (-143) elements
Clp0006I 0  Obj 10.186459 Primal inf 1.0225055e-05 (3) Dual inf 6.2075508e+13 (85)
Clp0029I End of values pass after 320 iterations
Clp0014I Perturbing problem by 0.001% of 3.2102504 - largest nonzero change 2.7825136e-05 ( 0.001575332%) - largest zero change 2.6625829e-05
Clp0000I Optimal - objective value 10.181818
Clp0000I Optimal - objective value 10.181818
Coin0511I After Postsolve, objective 10.181818, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 10.181818
Clp0000I Optimal - objective value 10.181818
Clp0000I Optimal - objective value 10.181818
Clp0000I Optimal - objective value 10.181818